In [1]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
import matplotlib.patches as mpatches

torch.set_default_dtype(torch.float64)
XDIM = 15
GAMING = 0.5
EPSILON = 0.1
SLOPE_C = 1
X_LOWER_BOUND = -4
X_UPPER_BOUND = 4

# Utils

In [2]:
def gen_data(N, informative_frac=1, shift_range=1, scale_range=1, noise_frac=0.01, seed=None):
    
    np.random.seed(seed)
    n_informative = informative_frac*XDIM
    n_redundant = XDIM - n_informative
    shift_arr = shift_range*np.random.randn(XDIM)
    scale_arr = scale_range*np.random.randn(XDIM)
    X, Y = make_classification(n_samples=N, n_features=XDIM, n_informative=n_informative, n_redundant=n_redundant,
                               flip_y=noise_frac, shift=shift_arr, scale=scale_arr, random_state=seed)
    Y[Y == 0] = -1
    return torch.from_numpy(X), torch.from_numpy(Y)
    
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    data = torch.cat((X, Y), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, :2]
    Y = data[:, 2]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def pred(X, w, b):
    return torch.sign(score(X, w, b))

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

def evaluate_model(X, Y, w, b, ccp, strategic):
    if not strategic:
        Xopt = X
    else:
        Xopt = ccp.optimize_X(X, w, b)
    Ypred = pred(Xopt, w, b)
    return calc_accuracy(Y, Ypred)

# CCP classes

In [3]:
class CCP:
    def __init__(self, funcs):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(XDIM)
        self.xt = cp.Parameter(XDIM)
        self.r = cp.Parameter(XDIM)
        self.w = cp.Parameter(XDIM)
        self.b = cp.Parameter(1)

        target = self.x@self.f_derivative(self.xt, self.w, self.b) - self.g(self.x, self.w, self.b) - self.c(self.x, self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(target), constraints)
        
    def ccp(self, r, w, b):
        """
        numpy to numpy
        """
        self.w.value = w
        self.b.value = b
        self.xt.value = r
        self.r.value = r
        
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        while diff > 0.0001:
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        X = X.numpy()
        return torch.stack([torch.from_numpy(self.ccp(x, w, b)) for x in X])

# Gain & Cost functions

In [4]:
v = np.array([-1,-1,-1,-1,-1,-1,-1,1,1,0.1,1,0.1,0.1,1,0.1])

def score(x, w, b):
    return x@w + b

def f(x, w, b):
    return 0.5*cp.norm(cp.hstack([1, (SLOPE_C*score(x, w, b) + 1)]), 2)

def g(x, w, b):
    return 0.5*cp.norm(cp.hstack([1, (SLOPE_C*score(x, w, b) - 1)]), 2)

def c_true(x, r):
    print(GAMING, EPSILON)
    return 2*(1./GAMING)*(EPSILON*cp.sum_squares(x-r) + (1-EPSILON)*cp.pos((x-r) @ v))

def c(x, r):
    return 2*(1./GAMING)*(cp.pos((x-r) @ v))

def f_derivative(x, w, b):
    return 0.5*SLOPE_C*((SLOPE_C*score(x, w, b) + 1)/cp.sqrt((SLOPE_C*score(x, w, b) + 1)**2 + 1))*w

def g_derivative(x, w, b):
    return 0.5*SLOPE_C*((SLOPE_C*score(x, w, b) - 1)/cp.sqrt((SLOPE_C*score(x, w, b) - 1)**2 + 1))*w

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "g_derivative": g_derivative, "c": c, "score": score}

# Data generation

In [5]:
def load_spam_dataset():
    path = r"C:\Users\sagil\Desktop\nir project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [6]:
X, Y = load_spam_dataset()
torch.manual_seed(1)
np.random.seed(1)
assert(len(X[0]) == XDIM)
X, Y, Xval, Yval = split_data(X, Y, 0.20)
print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

percent of positive samples: 49.44356120826709%


# Train

In [7]:
def fit(evaluate, loss, params, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, calc_train_errors=False):
    
    train_dset = TensorDataset(X, Y)
    train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
    opt = opt(params, **opt_kwargs)

    train_losses = []
    val_losses = []
    train_errors = []
    val_errors = []
    
    total_time = time.time()
    for epoch in range(epochs):
        t1 = time.time()
        
        batch = 1
        train_losses.append([])
        train_errors.append([])
        for Xbatch, Ybatch in train_loader:
            opt.zero_grad()
            l = loss(Xbatch, Ybatch)
            l.backward()
            opt.step()
            train_losses[-1].append(l.item())
            if calc_train_errors:
                with torch.no_grad():
                    e = evaluate(Xbatch, Ybatch)
                    train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | err: %3.5f" % 
                          (batch, len(train_loader), np.mean(train_losses[-1]), np.mean(train_errors[-1])))
            else:
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1])))
            batch += 1
            if callback is not None:
                callback()
                
        with torch.no_grad():
            val_losses.append(loss(Xval, Yval).item())
            val_errors.append(1-evaluate(Xval, Yval))
            
        t2 = time.time()
        if verbose:
            # print(t2-t1)
            print("----- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
    print("training time: {} seconds".format(time.time()-total_time)) 
    return train_errors, val_errors, train_losses, val_losses

def generate_delta_layer(funcs):
    g = funcs["g"]
    c = funcs["c"]
    
    x = cp.Variable(XDIM)
    w = cp.Parameter(XDIM, value = np.random.randn(XDIM))
    b = cp.Parameter(1, value = np.random.randn(1))
    r = cp.Parameter(XDIM, value = np.random.randn(XDIM))
    f_der = cp.Parameter(XDIM, value = np.random.randn(XDIM))

    target = x@f_der - g(x, w, b) - c(x, r)
    constraints = [x >= X_LOWER_BOUND,
                   x <= X_UPPER_BOUND]
    objective = cp.Maximize(target)
    problem = cp.Problem(objective, constraints)
    layer = CvxpyLayer(problem, parameters=[f_der, w, b, r], variables=[x])
    
    return layer

def get_f_ders(XT, w, b):
    """
    tensor to tensor
    """
    return torch.stack([0.5*SLOPE_C*((SLOPE_C*score(xt, w, b) + 1)/torch.sqrt((SLOPE_C*score(xt, w, b) + 1)**2 + 1))*w for xt in XT])


In [8]:
ccp = CCP(funcs)
delta_layer = generate_delta_layer(funcs)

def loss(X, Y, w, b, strategic=True):
    if strategic:
        XT = ccp.optimize_X(X, w, b)
        f_der = get_f_ders(XT, w, b)
        Xopt = delta_layer(f_der, w, b, X)[0] # Xopt should equal to XT but we do it again for the gradients
        output = score(Xopt, w, b)
        loss = torch.mean(torch.clamp(1 - output * Y, min=0))
    else:
        output = score(X, w, b)
        loss = torch.mean(torch.clamp(1 - output * Y, min=0))
    
    return loss


# Experiment

In [ ]:
EPOCHS = 7
BATCH_SIZE = 128

gaming_list = [0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3]
for t in gaming_list:
    failed = True
    while failed:
        try:
            failed = False
            print("training on t:{}".format(t))
            GAMING = t

            w_strategic = torch.zeros(XDIM, requires_grad=True)
            b_strategic = torch.zeros(1, requires_grad=True)

            train_errors, val_errors, train_losses, val_losses = fit(lambda X, Y: evaluate_model(X, Y, w_strategic, b_strategic, ccp, strategic=True), 
                                           lambda X, Y: loss(X, Y, w_strategic, b_strategic, strategic=True), [w_strategic, b_strategic], X, Y, Xval, Yval,
                                           opt=torch.optim.Adam, opt_kwargs={"lr": (1e-2)},
                                           batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=False, calc_train_errors=False)

            w_strategic.requires_grad = False
            b_strategic.requires_grad = False
        except:
            print("Failed")
            failed = True
        
    with open("classifiers_parameters.txt", "a+") as file:
        file.write("{}|{}|{}|{}|{}".format(t, w_strategic, b_strategic, val_errors[-1], val_losses[-1]))

training on t:0.3


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\diffcp\cone_program.py:259: UserWarning: Solved/Inaccurate.
  warnings.warn("Solved/Inaccurate.")


Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.3
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.3
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.3


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.3
training time: 2139.547792196274 seconds
training on t:0.6
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.6
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.6
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.6
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.6
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:0.6
Please consider re-formulating your problem so that it is always 

Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:3
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:3
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:3
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.
Failed
training on t:3
Failed
training on t:3


In [ ]:
# def add_ones(X):
#     n = X.size()[0]
#     ONES = torch.ones((n,1))
#     return torch.cat((X, ONES), 1)

# clf = svm.LinearSVC()

# clf.fit(add_ones(X), Y)
# w_svm = clf.coef_[0]
# b_svm = w_svm[-1]
# w_svm = w_svm[:-1]
# print(w_svm, b_svm)
# with open("classifiers_parameters.txt", "a+") as file:
#             file.write("svm|svm|{}|{}|NA|NA".format(w_svm, b_svm))

models = {}
with open("classifier_params.txt", "r+") as file:
    lines = file.readlines()

for line in lines:

    lst = line.split("|")
    gaming = lst[0]
    w = lst[1].replace("\n", "").split(",")
    w = list(map(float, w))
    b = [float(lst[2])]
    w = torch.Tensor(w)
    b = torch.Tensor(b)
    models[gaming] = (torch.Tensor(w), torch.Tensor(b))
    
funcs["c"] = c_true
gaming_list = [0.5, 1, 1.5, 2, 2.5, 3]
epsilon_list = [0.1, 0.3]
for e in epsilon_list:
    for t in gaming_list:
        print("evaluating ", e, t) 
        GAMING = t
        EPSILON = e
        ccp = CCP(funcs)
        our_w, our_b = models[str(t)]
        svm_w, svm_b = models['svm']
        our_accuracy = evaluate_model(Xval, Yval, our_w, our_b, ccp, strategic=True)
        svm_accuracy = evaluate_model(Xval, Yval, svm_w, svm_b, ccp, strategic=True)
        
        with open("experiment_results.txt", "a+") as file:
            file.write("ours|{}|{}|{}\n".format(e, t, our_accuracy))
            file.write("svm|{}|{}|{}\n".format(e, t, svm_accuracy))
